### Identify Long Homopolymer Areas on Fasta

In [1]:
def region_toks(region):
    return region.replace(":", "\t").replace("-", "\t").split("\t")

def region_chr(region):
    return region_toks[0]

def region_start(region):
    return int(region_toks(region)[1])

def region_end(region):
    return int(region_toks(region)[2])

In [2]:
import io
import subprocess
import re

# extract bases 
def get_bases(ref, region):
    cmd = "bedtools getfasta -fi %s -bed -" % ref
    beddata = "%s\t%s\t%s\n" % tuple(region_toks(region))
    start = region_start(region)
    with subprocess.Popen(cmd, stdout=subprocess.PIPE, stdin=subprocess.PIPE, shell=True) as proc:
        stdout = proc.communicate(input=beddata.encode())[0]
        bases = stdout.decode().split('\n')[1].upper()
        return (start, bases)


In [3]:
import re

# find homopolymer areas
def get_homopolymer_spans(start, bases, hmerSize):
    patt = re.compile("(T{%d,})|(A{%d,})|(G{%d,})|(C{%d,})" % (hmerSize, hmerSize, hmerSize, hmerSize))
    spans = [(x.span()[0] + start, x.span()[1] + start - 1) for x in re.finditer(patt, bases)]
    return spans

#spans = get_homopolymer_spans(start, bases, hmerSize)
#spans

### get alleles from a vcf file

In [4]:
# params
vcf = "output/test1/baseline/test1-baseline.vcf"

def get_vcf_alleles(vcf):
    cmd = "grep -E -v '^#' %s | cut -f 2,4,5" % vcf
    with subprocess.Popen(cmd, stdout=subprocess.PIPE, stdin=subprocess.PIPE, shell=True) as proc:
        stdout = proc.communicate(input="".encode())[0]
        alleles = [form_allele(x.split("\t")) for x in stdout.decode().split('\n')[:-1]]
        return alleles

def form_allele(a):
    start = int(a[0])
    length = len(a[1])
    return (start, start+length-1)
    
#alleles = get_vcf_alleles(vcf)
#alleles[0:12]

In [5]:
def span_complement(span, start, end):
    all = sum([list(x) for x in span], [])
    comp = []
    comp.append((start, all[0] - 1))
    it = iter(all[1:-1])
    for x in it:
        comp.append((x + 1, next(it) - 1))
    comp.append((all[-1] + 1, end))
    return comp

#non_homo_spans = span_complement(homo_spans, region_start(region), region_end(region))
#non_homo_spans[0:12]


In [6]:
import bisect

def loc_in_spans(spans, loc):
    locs = sum([list(x) for x in spans], [])
    return loc_in_flatten_spans(locs, loc)

def loc_in_flatten_spans(f_spans, loc):
    i = bisect.bisect_left(f_spans, loc)
    if i < len(f_spans) and f_spans[i] == loc:
        return True
    else:
        return bool(i & 1)
    

def alleles_in_spans(spans, alleles):
    locs = sum([list(x) for x in spans], [])
    return [x for x in alleles if loc_in_flatten_spans(locs, x[0])]
    
    
#print(loc_in_spans(homo_spans, 6484073))

In [8]:
spans_db = {}
def summarize_test(tname, tver, hmerSize, ref, region):

    # make sure we have bases
    if not ref in spans_db:
        spans_db[ref] = {}
    if not region in spans_db[ref]:
        spans_db[ref][region] = get_bases(ref, region)[1]
    
    # make sure we have spans
    if not hmerSize in spans_db:
        spans_db[hmerSize] = {}
        spans_db[hmerSize]["homo_spans"] = get_homopolymer_spans(region_start(region), spans_db[ref][region], hmerSize)
        spans_db[hmerSize]["non_homo_spans"] = span_complement(spans_db[hmerSize]["homo_spans"], region_start(region), region_end(region))
        #print("hmerSize %d homo_spans %d" % (hmerSize, len(spans_db[hmerSize]["homo_spans"])))
    
    # get alleles
    vcf = "output/{0}/{1}/{0}-{1}.vcf".format(tname, tver)
    info = {}
    info["alleles"] = get_vcf_alleles(vcf)

    # map to homo/non-homo
    info["homo_alleles"] = alleles_in_spans(spans_db[hmerSize]["homo_spans"], info["alleles"])
    info["non_homo_alleles"] = alleles_in_spans(spans_db[hmerSize]["non_homo_spans"], info["alleles"])
    
    info["lengths"] = "%d alleles %d homo_alleles %d non_homo_alleles" % (
            len(info["alleles"]),
            len(info["homo_alleles"]),
            len(info["non_homo_alleles"]),
        )
    
    info["row"] = {
        "region": region,
        "hmerSize": hmerSize,
        "homo_spans": len(spans_db[hmerSize]["homo_spans"]),
        "tname": tname,
        "tver": tver,
        "alleles": len(info["alleles"]),
        "homo_alleles": len(info["homo_alleles"]),
        "non_homo_alleles": len(info["non_homo_alleles"]),
    }
    return info
    
def check_info(info):
    # convert lists of alleles to dictionaries with start as the key
    dicts = {}
    for name in ["alleles", "homo_alleles", "non_homo_alleles"]:
        dicts[name] = {x[0]:x for x in info[name]}
        
    # make sure home and non_homo are disjoined
    h_keys = set(dicts["homo_alleles"].keys())
    nh_keys = set(dicts["non_homo_alleles"].keys())
    nh_h_keys = h_keys.intersection(nh_keys)
    #print("h_keys %d nh_keys %d nh_h_keys %d" % (len(h_keys), len(nh_keys), len(nh_h_keys)))
    if len(nh_h_keys):
        print("ERROR: homo and non_homo sets intersect!")
        print(nh_h_keys)
    
    # make sure all homo and non_homo are in alleles
    a_keys = set(dicts["alleles"].keys())
    a_h_keys = a_keys.intersection(h_keys)
    #print("a_keys %d h_keys %d a_h_keys %d" % (len(a_keys), len(h_keys), len(a_h_keys)))
    if len(h_keys) != len(a_h_keys):
        print("ERROR: allels and homo sets disjointed!")
    a_nh_keys = a_keys.intersection(nh_keys)
    #print("a_keys %d nh_keys %d a_nh_keys %d" % (len(a_keys), len(nh_keys), len(a_nh_keys)))
    if len(nh_keys) != len(a_nh_keys):
        print("ERROR: allels and non-homo sets disjointed!")
    
    # make sure homo+non-homo=alleles
    diff_keys = a_keys.difference(h_keys.union(nh_keys))
    if len(diff_keys):
        print("ERROR: %d keys missing from homo+non-homo!" % len(diff_keys))
        for key in diff_keys:
            print(dicts["alleles"][key])
    
    
    
tests = {
            "test1": [
                {
                    "hmerSize": 12,
                    "region": "chr20:6000000-7000000",
                    "ref": "../ref/Homo_sapiens_assembly38.fasta",
                    "versions": ["baseline", "baseline12m", "collapse12", "collapse12m"]
                },
                {
                    "hmerSize": 10,
                    "region": "chr20:6000000-7000000",
                    "ref": "../ref/Homo_sapiens_assembly38.fasta",
                    "versions": ["baseline", "baseline10m", "collapse10", "collapse10m"]
                }
            ],
            "test1p": [
                {
                    "hmerSize": 12,
                    "region": "chr20:6000000-7000000",
                    "ref": "../ref/Homo_sapiens_assembly38.fasta",
                    "versions": ["baseline", "baseline12m", "collapse12", "collapse12m"]
                },
                {
                    "hmerSize": 10,
                    "region": "chr20:6000000-7000000",
                    "ref": "../ref/Homo_sapiens_assembly38.fasta",
                    "versions": ["baseline", "baseline10m", "collapse10", "collapse10m"]
                }
            ]
        }

spans_db["info"] = {}
df_rows = []
for tname,tdicts in tests.items():
    spans_db["info"][tname] = {}
    for tdict in tdicts:
        for tver in tdict["versions"]:
            info = summarize_test(tname, tver, tdict["hmerSize"], tdict["ref"], tdict["region"])
            spans_db["info"][tname][tver] = info
            df_rows.append(info["row"])
            #print("%s-%s: %s" % (tname, tver, info["lengths"]))
            check_info(info)
            
import pandas as pd
info_df = pd.DataFrame(df_rows)
info_df

,region,hmerSize,homo_spans,tname,tver,alleles,homo_alleles,non_homo_alleles
0,chr20:6000000-7000000,12,205,test1,baseline,3071,208,2863
1,chr20:6000000-7000000,12,205,test1,baseline12m,2957,96,2861
2,chr20:6000000-7000000,12,205,test1,collapse12,3043,185,2858
3,chr20:6000000-7000000,12,205,test1,collapse12m,2893,35,2858
4,chr20:6000000-7000000,10,320,test1,baseline,3071,308,2763
5,chr20:6000000-7000000,10,320,test1,baseline10m,2847,115,2732
6,chr20:6000000-7000000,10,320,test1,collapse10,351,80,271
7,chr20:6000000-7000000,10,320,test1,collapse10m,272,7,265
8,chr20:6000000-7000000,12,205,test1p,baseline,3089,209,2880
9,chr20:6000000-7000000,12,205,test1p,baseline12m,2971,95,2876
